In [1]:
!pip install -U coremltools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.0 MB/s eta 0:00:00


In [2]:
import urllib
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import torch
import torch.nn as nn
import torchvision
import json

from torchvision import transforms
from PIL import Image

import coremltools as ct

In [3]:
class WrappedDeeplabv3Resnet101(nn.Module):

    def __init__(self):
        super(WrappedDeeplabv3Resnet101, self).__init__()
        self.model = torch.hub.load(
            "pytorch/vision:v0.6.0",
            "deeplabv3_resnet101",
            pretrained=True
        ).eval()

    def forward(self, input):
        res = self.model(input)
        # Extract the tensor we want from the output dictionary
        x = res["out"]
        return x

In [6]:
#tracer input
from PIL import Image
import requests
from io import BytesIO

img_url="https://apple.github.io/coremltools/docs-guides/_images/cat_dog.jpg"
response = requests.get(img_url)
input_img = Image.open(BytesIO(response.content))

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

input_tensor = preprocess(input_img)
input_batch = input_tensor.unsqueeze(0)

In [7]:
torch_model = WrappedDeeplabv3Resnet101().eval()
traced_model = torch.jit.trace(torch_model, input_batch)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
mlmodel_from_trace = ct.convert(
    traced_model,
    inputs=[ct.TensorType(name="input", shape=input_batch.shape)],
)

Running MIL default pipeline:   9%|▉         | 8/89 [00:00<00:02, 36.17 passes/s]/usr/local/lib/python3.11/dist-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:273: UserWarning: Output, '1683', of the source model, has been renamed to 'var_1683' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 49.48 passes/s]


In [9]:
mlmodel_from_trace.save("/content/SegmentationModel_no_metadata_from_trace.mlpackage")


In [11]:
# load the model
mlmodel = ct.models.MLModel("/content/SegmentationModel_no_metadata_from_trace.mlpackage")

labels_json = {"labels": ["background", "aeroplane", "bicycle", "bird", "board", "bottle", "bus", "car", "cat", "chair", "cow", "diningTable", "dog", "horse", "motorbike", "person", "pottedPlant", "sheep", "sofa", "train", "tvOrMonitor"]}

mlmodel.user_defined_metadata["com.apple.coreml.model.preview.type"] = "imageSegmenter"
mlmodel.user_defined_metadata["com.apple.coreml.model.preview.params"] = json.dumps(labels_json)

mlmodel.save("/content/SegmentationModel_with_metadata.mlpackage")

In [12]:
from google.colab import files
files.download('/content/SegmentationModel_with_metadata.mlpackage')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>